# 迁移学习对比实验
本 notebook 演示迁移学习（使用预训练权重）与从头训练在 Fashion-MNIST 数据集上的性能对比。

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models
from torch.utils.data import Subset
import numpy as np
import matplotlib.pyplot as plt
from torchsummary import summary

batch_size = 16
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"使用设备: {device}")

## 1. 数据准备
- Fashion-MNIST 3x299x299
- 仅取 10% 的数据

In [ ]:
transform = transforms.Compose([
    transforms.Resize(299),
    transforms.Grayscale(num_output_channels=3),
    transforms.ToTensor()
])

train_full = datasets.FashionMNIST('data', train=True, download=True, transform=transform)
test_full = datasets.FashionMNIST('data', train=False, download=True, transform=transform)

n = 10  # 选取 1/10 的数据
rng = np.random.default_rng(42)
train_idx = rng.choice(len(train_full), len(train_full) // n, replace=False)
test_idx = rng.choice(len(test_full), len(test_full) // n, replace=False)

train_loader = torch.utils.data.DataLoader(Subset(train_full, train_idx),
                                           batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(Subset(test_full, test_idx),
                                          batch_size=batch_size, shuffle=False)

print(f"训练样本数: {len(train_loader.dataset)}")
print(f"测试样本数: {len(test_loader.dataset)}")

## 2. 训练函数定义

In [ ]:
def train_model(use_pretrained=True):
    """训练模型并返回测试结果"""
    if use_pretrained:
        print("\n" + "=" * 60)
        print("训练模型：使用预训练权重（迁移学习）")
        print("=" * 60)
        model = models.inception_v3(weights=models.Inception_V3_Weights.DEFAULT)
        lr = 1e-4  # 迁移学习使用较小的学习率
    else:
        print("\n" + "=" * 60)
        print("训练模型：不使用预训练权重（从头训练）")
        print("=" * 60)
        model = models.inception_v3(weights=None)  # 不加载预训练权重
        lr = 1e-3  # 从头训练使用较大的学习率

    model.fc = nn.Linear(model.fc.in_features, 10)
    model = model.to(device)
    optimizer = optim.Adam(model.parameters(), lr=lr)

    # 打印模型结构
    print(f"\n模型结构摘要:")
    summary(model, input_size=(3, 299, 299))

    # 训练/测试
    accs, losses = [], []
    epochs = 10

    for epoch in range(epochs):
        # 训练阶段
        model.train()
        for x, y in train_loader:
            x, y = x.to(device), y.to(device)
            optimizer.zero_grad()

            # Inception-v3 在训练时返回辅助输出和主要输出
            if model.training:
                out, aux_out = model(x)
            else:
                out = model(x)

            loss = F.cross_entropy(out, y)
            loss.backward()
            optimizer.step()

        # 测试阶段
        model.eval()
        with torch.no_grad():
            correct, total_loss = 0, 0.
            test_total = 0

            for x, y in test_loader:
                x, y = x.to(device), y.to(device)
                out = model(x)  # 在eval模式下不返回辅助输出
                total_loss += F.cross_entropy(out, y).item()
                _, predicted = out.max(1)
                correct += predicted.eq(y).sum().item()
                test_total += y.size(0)

        acc = correct / test_total
        avg_loss = total_loss / len(test_loader)
        accs.append(acc)
        losses.append(avg_loss)

        print(f'epoch {epoch}: loss={avg_loss:.4f}, acc={acc:.4f}')

    return accs, losses, model

## 3. 模型训练对比

In [ ]:
# 训练不使用预训练权重的模型
accs_no_pretrain, losses_no_pretrain, model_no_pretrain = train_model(use_pretrained=False)

In [ ]:
# 训练使用预训练权重的模型
accs_pretrain, losses_pretrain, model_pretrain = train_model(use_pretrained=True)

## 4. 可视化对比结果

In [ ]:
plt.figure(figsize=(14, 5))

# Loss 曲线对比
plt.subplot(1, 2, 1)
plt.plot(losses_pretrain, marker='o', color='red', linewidth=2, label='With Pretrained Weights')
plt.plot(losses_no_pretrain, marker='s', color='green', linewidth=2, label='Without Pretrained Weights')
plt.title("Test Loss Curve Comparison", fontsize=14, fontweight='bold')
plt.xlabel("Epoch", fontsize=12)
plt.ylabel("Loss", fontsize=12)
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.xticks(range(len(losses_pretrain)))

# Accuracy 曲线对比
plt.subplot(1, 2, 2)
plt.plot(accs_pretrain, marker='o', color='red', linewidth=2, label='With Pretrained Weights')
plt.plot(accs_no_pretrain, marker='s', color='green', linewidth=2, label='Without Pretrained Weights')
plt.title("Test Accuracy Curve Comparison", fontsize=14, fontweight='bold')
plt.xlabel("Epoch", fontsize=12)
plt.ylabel("Accuracy", fontsize=12)
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.xticks(range(len(accs_pretrain)))

plt.tight_layout()
plt.savefig("res/迁移学习对比.png", dpi=150, bbox_inches='tight')
plt.show()

## 5. 结果分析

In [ ]:
print("=" * 60)
print("迁移学习对比实验结果摘要")
print("=" * 60)
print(f"\n使用预训练权重模型最终结果:")
print(f"  - 最终准确率: {accs_pretrain[-1]:.4f} ({accs_pretrain[-1]*100:.2f}%)")
print(f"  - 最终损失: {losses_pretrain[-1]:.4f}")
print(f"  - 最佳准确率: {max(accs_pretrain):.4f} ({max(accs_pretrain)*100:.2f}%) 第{np.argmax(accs_pretrain)+1}轮")

print(f"\n从头训练模型最终结果:")
print(f"  - 最终准确率: {accs_no_pretrain[-1]:.4f} ({accs_no_pretrain[-1]*100:.2f}%)")
print(f"  - 最终损失: {losses_no_pretrain[-1]:.4f}")
print(f"  - 最佳准确率: {max(accs_no_pretrain):.4f} ({max(accs_no_pretrain)*100:.2f}%) 第{np.argmax(accs_no_pretrain)+1}轮")

print(f"\n性能提升:")
improvement = (accs_pretrain[-1] - accs_no_pretrain[-1]) * 100
print(f"  - 准确率提升: {improvement:.2f}%")
print("=" * 60)

## 6. 模型保存

In [ ]:
# 保存训练好的模型
torch.save({
    'model_pretrain_state_dict': model_pretrain.state_dict(),
    'model_no_pretrain_state_dict': model_no_pretrain.state_dict(),
    'accs_pretrain': accs_pretrain,
    'losses_pretrain': losses_pretrain,
    'accs_no_pretrain': accs_no_pretrain,
    'losses_no_pretrain': losses_no_pretrain
}, 'transfer_learning_comparison.pth')

print("模型已保存为 'transfer_learning_comparison.pth'")